**Load data**

Load from csv into pandas. Each sequence, one by one.

In [1]:
import pandas as pd

from util.data import load_data, preprocess, make_dataset, split_train_test_validation

assets = load_data('data')

symbol = next(iter(assets.keys()))
asset = assets[symbol]
date_time = pd.to_datetime(asset.pop('date'), format='%Y-%m-%d')

plot_cols = ['close', 'volume']
plot_features = asset[plot_cols]
plot_features.index = date_time
_ = plot_features.plot(subplots=True)

asset.describe().transpose()

train, validation, test = split_train_test_validation(asset)
train = make_dataset(train, sequence_length=60, sequence_stride=1, shift=1)
validation = make_dataset(validation, sequence_length=60, sequence_stride=1, shift=1)
test = make_dataset(test, sequence_length=60, sequence_stride=1, shift=1)

print('train', train)
print('validation', validation)
print('test', test)

1650
0 (32, 59, 6) (32, 59, 1) 2
1 (32, 59, 6) (32, 59, 1) 2
2 (32, 59, 6) (32, 59, 1) 2
3 (32, 59, 6) (32, 59, 1) 2
4 (32, 59, 6) (32, 59, 1) 2
5 (32, 59, 6) (32, 59, 1) 2
6 (32, 59, 6) (32, 59, 1) 2
7 (32, 59, 6) (32, 59, 1) 2
8 (32, 59, 6) (32, 59, 1) 2
9 (32, 59, 6) (32, 59, 1) 2
10 (32, 59, 6) (32, 59, 1) 2
11 (32, 59, 6) (32, 59, 1) 2
12 (32, 59, 6) (32, 59, 1) 2
13 (32, 59, 6) (32, 59, 1) 2
14 (32, 59, 6) (32, 59, 1) 2
15 (32, 59, 6) (32, 59, 1) 2
16 (32, 59, 6) (32, 59, 1) 2
17 (32, 59, 6) (32, 59, 1) 2
18 (32, 59, 6) (32, 59, 1) 2
19 (32, 59, 6) (32, 59, 1) 2
20 (32, 59, 6) (32, 59, 1) 2
21 (32, 59, 6) (32, 59, 1) 2
22 (32, 59, 6) (32, 59, 1) 2
23 (32, 59, 6) (32, 59, 1) 2
24 (32, 59, 6) (32, 59, 1) 2
25 (32, 59, 6) (32, 59, 1) 2
26 (32, 59, 6) (32, 59, 1) 2
27 (32, 59, 6) (32, 59, 1) 2
28 (32, 59, 6) (32, 59, 1) 2
29 (32, 59, 6) (32, 59, 1) 2
30 (32, 59, 6) (32, 59, 1) 2
31 (32, 59, 6) (32, 59, 1) 2
32 (32, 59, 6) (32, 59, 1) 2
33 (32, 59, 6) (32, 59, 1) 2
34 (32, 59, 6) (32,

**Strategies**
The ML models are evaluated using the following strategies:
* Average return
* Sharpe ratio

**Model**

Initialize Model

In [7]:
import tensorflow as tf
from tensorflow import keras

MAX_EPOCHS = 100

def returns_loss(y_true, y_pred):
    return 0.1

def compile_and_fit(model, train, val, patience=10):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=returns_loss,
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])
  model.summary()
  history = model.fit(train, epochs=MAX_EPOCHS,
                      validation_data=val,
                      callbacks=[early_stopping])
  return history

lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(50, return_sequences=True),
    tf.keras.layers.Dense(units=1)
])

print('Input shape:', next(iter(train))[0].shape)
print('Output shape:', lstm_model(next(iter(train))[0]).shape)

history = compile_and_fit(lstm_model, train, validation)

Input shape: (32, 59, 6)
Output shape: (32, 59, 1)
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (32, 59, 50)              11400     
_________________________________________________________________
dense_5 (Dense)              (32, 59, 1)               51        
Total params: 11,451
Trainable params: 11,451
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100


ValueError: in user code:

    C:\Users\Johan\envs\machinelearning\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\Users\Johan\envs\machinelearning\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\Johan\envs\machinelearning\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\Johan\envs\machinelearning\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\Johan\envs\machinelearning\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\Johan\envs\machinelearning\lib\site-pack

**Training**

Run training.